In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import random
import h5py
import math

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving test.hdf5 to test.hdf5
Saving train.hdf5 to train.hdf5


In [ ]:
f1 = h5py.File("train.hdf5", 'r')
X_train, Y_train = f1['image'][...], f1['label'][...]

f2 = h5py.File("test.hdf5", 'r')
X_test, Y_test = f2['image'][...], f2['label'][...]

In [ ]:
X_train = np.array(f1['image'])/255
Y_train = np.array([f1['label']])
print(X_train.shape)
print(Y_train.shape)

X_test = np.array(f2['image'])/255
Y_test = np.array([f2['label']])
print(X_test.shape)
print(Y_test.shape)


(60000, 28, 28)
(1, 60000)
(10000, 28, 28)
(1, 10000)


In [ ]:
X_train = np.reshape(X_train,(60000,784)).T
print(X_train.shape)
print(Y_train.shape)

X_test = np.reshape(X_test,(10000,784)).T
print(X_test.shape)
print(Y_test.shape)


(784, 60000)
(1, 60000)
(784, 10000)
(1, 10000)


In [ ]:
n_X=X_train.shape[0]
m=X_train.shape[1]

In [ ]:
shape = (Y_train.max()+1, Y_train.shape[1]) #Generating one_hot_vector so that later we can apply multi class classification
one_hot = np.zeros(shape)
rows= np.arange(Y_train.shape[1])
one_hot[Y_train , rows]=1
Y_train = one_hot

In [ ]:
shape = (Y_test.max()+1, Y_test.shape[1]) #Generating one_hot_vector so that later we can apply multi class classification
one_hot = np.zeros(shape)
rows= np.arange(Y_test.shape[1])
one_hot[Y_test , rows]=1
Y_test = one_hot

In [ ]:
print(Y_train.shape)
print(Y_test.shape)

(10, 60000)
(10, 10000)


In [ ]:
n_X=X_train.shape[0]
m=X_train.shape[1]

def softmax(A):               #Softmax gives a good classification as it standardizes the output values.
    A = A - np.max(A)
    return np.exp(A) / np.sum(np.exp(A),axis=0) 

nums = np.array([4, 5, 6])
print(softmax(nums))

[0.09003057 0.24472847 0.66524096]


In [ ]:
#def random_mini_batches(X,Y,mini_bactch_size=64):
m = X_train.shape[1] 
mini_batches = []

permutation = list(np.random.permutation(m))
shuffled_X = X_train[:,permutation]
shuffled_Y = Y_train[:,permutation]

print(shuffled_X.shape)
print(shuffled_Y.shape)

(784, 60000)
(10, 60000)


In [ ]:
def random_mini_batches(X,Y,mini_batch_size=64):
  m = X.shape[1] 
  mini_batches = []

  permutation = list(np.random.permutation(m))
  shuffled_X = X [:,permutation]
  shuffled_Y = Y [:,permutation]

  num_comp_batchs = math.floor (m/mini_batch_size)

  for i in range (num_comp_batchs):
    mini_batch_X = shuffled_X [:, i*mini_batch_size : (i+1)*mini_batch_size ]
    mini_batch_Y = shuffled_Y [: ,i*mini_batch_size : (i+1)*mini_batch_size ]

    mini_batch = ( mini_batch_X, mini_batch_Y )
    mini_batches.append(mini_batch)

  if m % mini_batch_size != 0:
    end = m - mini_batch_size*math.floor(m/mini_batch_size)
    mini_batch_X = shuffled_X [:, num_comp_batchs*mini_batch_size : ]
    mini_batch_Y = shuffled_Y [:, num_comp_batchs*mini_batch_size : ]

    mini_batch =( mini_batch_X, mini_batch_Y )
    mini_batches.append(mini_batch)

  return mini_batches

  

In [ ]:
#initialize_parameters
layer_dims=[784,600,400,200,100,50,10]

parameters={}
L=len(layer_dims)
for l in range(1,L):
  parameters["W"+str(l)]= np.random.randn(layer_dims[l],layer_dims[l-1])*np.sqrt(2/layer_dims[l-1])

  parameters["b"+str(l)]= np.zeros((layer_dims[l],1))*0.01


def DNN_model(X,Y,parameters,layer_dims,num_iterations,learning_rate):

  m=X.shape[1]
  Y_prediction=np.zeros((1,m))
  forward_cache={}
  
  
  L=len(layer_dims)
  grads={}
  costs=[]
  cost_1=1000
  cost_2=1000
  for i in range(0,num_iterations):
    mini_batches = random_mini_batches(X,Y,mini_batch_size=64)

    for mini_batch in mini_batches:
      (mini_batch_X,mini_batch_Y) = mini_batch
      forward_cache["A0"]=np.copy(mini_batch_X)
    
      A=np.copy(mini_batch_X)
      for l in range(1,L-1):
      
        A_prev=np.copy(A)
      
        forward_cache["Z"+str(l)]=np.dot(parameters["W"+str(l)],A_prev)+parameters["b"+str(l)]
        A=np.maximum(0,forward_cache["Z"+str(l)])
      
        forward_cache["A"+str(l)]=np.copy(A)
    #last_layer
      W=parameters["W"+str(L-1)]
      b=parameters["b"+str(L-1)]
      forward_cache["Z"+str(L-1)]=np.dot(W,A)+b
      AL_train=  softmax(forward_cache["Z"+str(L-1)])
    #compute_cost
      m=mini_batch_Y.shape[1]
      print(mini_batch_Y.shape,AL_train.shape)
      cost_1 = -np.mean(mini_batch_Y * np.log(AL_train + 1e-8))

    #print(np.sum(W))
    #print(np.sum(b))
      print(cost_1)

      costs.append(float(cost_1))
    #backward_prop
      grads["dZ"+str(L-1)]=AL_train-mini_batch_Y
      grads["dW"+str(L-1)]=(1/m)*(np.dot(grads["dZ"+str(L-1)],forward_cache["A"+str(L-2)].T))
      grads["db"+str(L-1)]=(1/m)*(np.sum(grads["dZ"+str(L-1)],axis=1,keepdims=True))
      for l in reversed(range(1,L-1)):
        grads["dZ"+str(l)]=np.multiply(np.dot(parameters["W"+str(l+1)].T,grads["dZ"+str(l+1)]),np.int64(forward_cache["A"+str(l)]>0))
        grads["dW"+str(l)]=(1/m)*(np.dot(grads["dZ"+str(l)],(forward_cache["A"+str(l-1)]).T))
        grads["db"+str(l)]=(1/m)*(np.sum(grads["dZ"+str(l)],axis=1,keepdims=True))
      for l in range(1, L):
        parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate * grads["dW" + str(l)]
        parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate * grads["db" + str(l)]
     
  return parameters,costs,AL_train

In [ ]:
parameters,costs,AL_train= DNN_model(X_train,Y_train,parameters,layer_dims,num_iterations=5,learning_rate=0.1)
num_iterations=5

plt.plot(list(range(len(costs))),list(costs))
plt.show()

In [ ]:
print(AL_train.shape)

(10, 32)


In [ ]:


def DNN_model_batch(X,Y,parameters):

  m=X.shape[1]
  Y_prediction=np.zeros((1,m))
  forward_cache={}
  
  forward_cache["A0"]=np.copy(X)
  L=len(layer_dims)
  grads={}
  costs=[]
  cost_1=1000
  cost_2=1000
  for i in range(0,num_iterations):
    
    A=np.copy(X)
    for l in range(1,L-1):
      
      A_prev=np.copy(A)
      
      forward_cache["Z"+str(l)]=np.dot(parameters["W"+str(l)],A_prev)+parameters["b"+str(l)]
      A=np.maximum(0,forward_cache["Z"+str(l)])
      
      forward_cache["A"+str(l)]=np.copy(A)
    #last_layer
    W=parameters["W"+str(L-1)]
    b=parameters["b"+str(L-1)]
    forward_cache["Z"+str(L-1)]=np.dot(W,A)+b
    AL_train=  softmax(forward_cache["Z"+str(L-1)])
     
  return parameters,AL_train

In [ ]:
  for i in range(AL_train.shape[1]):
    
    a = np.max(AL_train[:,i])
    AL_train[:,i] = (AL_train[:,i]==a)
  fai = np.absolute(AL_train - Y_train)
  count = 0
  for i in range(0,fai.shape[1]):
    if np.sum(fai[:,i]) == 0:
      count +=1
  print(count)
  print((count /X_train.shape[1])*100)

In [ ]:
#for test set
parameters,AL_test= DNN_model_batch(X_test,Y_test,parameters)

In [ ]:
print(AL_test.shape)

(10, 10000)


In [ ]:
  for i in range(AL_test.shape[1]):
    
    a = np.max(AL_test[:,i])
    AL_test[:,i] = (AL_test[:,i]==a)
  fai = np.absolute(AL_test - Y_test)
  count = 0
  for i in range(0,fai.shape[1]):
    if np.sum(fai[:,i]) == 0:
      count +=1
  print(count)
  print((count /X_test.shape[1])*100)

9688
96.88
